# Data Cleaning - Running Metadata of the RICKD Analysis

In this notebook we will focus on cleaning the run_data_meta.csv file with what we have learned from the previous notebook.

- Coalesce `SpecInjury` and `SpecInjury2` into a single column as the main injury.
- Consolidating categorical data (i.e. Other/Other, Plantar fasciitis/Plantar fasciitis)
- Removing duplicates if they exist.
- Clean free text fields.
- Consolidate data from multiple sessions for the same subject.
- Some subjects have been registered under the same ID but they are different people. i.e. 200375
- Investigate outliers in the data.
- Investigate missing data.

# Output datasets:
- run_data_meta_cleaned.csv

In [2]:
import pandas as pd
from core.constants import RICKD_RUNNING_METADATA_FILE
from core.data_quality import identify_conflicting_data

In [3]:
run_data_meta = pd.read_csv(RICKD_RUNNING_METADATA_FILE)

In [6]:
# 1. Coalesce SpecInjury and SpecInjury2 into a single column as the main injury
run_data_meta['MainInjury'] = run_data_meta['SpecInjury'].combine_first(run_data_meta['SpecInjury2'])
run_data_meta['MainInjury']

0                      pain
1         disc degeneration
2                     other
3                       NaN
4                       NaN
               ...         
1827                    NaN
1828    Pelvic malalignment
1829          Muscle strain
1830                    NaN
1831                  Other
Name: MainInjury, Length: 1832, dtype: object

In [ ]:
# 1. Coalesce SpecInjury and SpecInjury2 into a single column as the main injury
run_data_meta['MainInjury'] = run_data_meta['SpecInjury'].combine_first(run_data_meta['SpecInjury2'])

# 2. Consolidate categorical data (e.g., 'Other/Other', 'Plantar fasciitis/Plantar fasciitis')
def consolidate_categorical(val):
    if isinstance(val, str) and '/' in val:
        parts = [p.strip() for p in val.split('/')]
        unique_parts = list(dict.fromkeys(parts))  # Remove duplicates, preserve order
        return '/'.join(unique_parts)
    return val

categorical_cols = ['MainInjury']  # Add other categorical columns as needed
for col in categorical_cols:
    run_data_meta[col] = run_data_meta[col].apply(consolidate_categorical)


# 3. Remove duplicates if they exist
run_data_meta = run_data_meta.drop_duplicates()


# 4. Clean free text fields
free_text_cols = ['MainInjury']  # Add other free text columns as needed
for col in free_text_cols:
    run_data_meta[col] = run_data_meta[col].astype(str).str.strip().str.title()



# 5. Consolidate data from multiple sessions for the same subject
# Example assumes 'SubjectID' column exists
agg_funcs = {
    'MainInjury': lambda x: '/'.join(sorted(set(x.dropna())))
    # Add other columns and aggregation functions as needed
}
if 'SubjectID' in run_data_meta.columns:
    run_data_meta = run_data_meta.groupby('SubjectID', as_index=False).agg(agg_funcs)



# 6. Handle subjects with the same ID but are different people (e.g., 200375)
# Manual step: If you know which rows are different people, assign new IDs
# Example: run_data_meta.loc[run_data_meta.index == problematic_row_index, 'SubjectID'] = '200375_A'
# (You need to specify which rows are which person.)




# 7. Investigate outliers in the data
display(run_data_meta.describe(include='all'))
# You can also use boxplots for visualization if needed
# import matplotlib.pyplot as plt
# run_data_meta.boxplot(rot=90)
# plt.show()



# 8. Investigate missing data
display(run_data_meta.isnull().sum())
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(run_data_meta.isnull(), cbar=False)
plt.show()

In [5]:
run_data_meta.describe(include='all')

,sub_id,datestring,filename,speed_r,age,Height,Weight,Gender,DominantLeg,InjDefn,...,SpecInjury2,Activities,Level,YrsRunning,RaceDistance,RaceTimeHrs,RaceTimeMins,RaceTimeSecs,YrPR,NumRaces
count,1832.000000,1832,1832,1832.000000,1832.000000,1829.000000,1830.000000,1832,1480,1752,...,320,1516,1563,1315.000000,1494,979,1025,924,425.000000,504.000000
unique,NaN,1823,1832,NaN,NaN,NaN,NaN,3,3,4,...,55,800,2,NaN,9,18,70,47,NaN,NaN
top,NaN,NaT,20101005T132240.json,NaN,NaN,NaN,NaN,Female,Right,No injury,...,pain,running,Recreational,NaN,Casual Runner (no times),HH,MM,SS,NaN,NaN
freq,NaN,10,1,NaN,NaN,NaN,NaN,926,1131,659,...,74,153,1042,NaN,436,424,324,479,NaN,NaN
mean,122721.658843,NaN,NaN,2.760160,38.170306,173.051919,71.017223,NaN,NaN,NaN,...,NaN,NaN,NaN,49.122624,NaN,NaN,NaN,NaN,1930.315294,5.329365
std,41154.448668,NaN,NaN,0.477627,13.145301,29.675143,37.466057,NaN,NaN,NaN,...,NaN,NaN,NaN,191.117688,NaN,NaN,NaN,NaN,394.518125,5.480369
min,100001.000000,NaN,NaN,1.172048,18.000000,0.000000,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
25%,100608.750000,NaN,NaN,2.482615,28.000000,165.200000,60.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,3.250000,NaN,NaN,NaN,NaN,2012.000000,2.000000
50%,101256.000000,NaN,NaN,2.721310,37.000000,172.700000,69.100000,NaN,NaN,NaN,...,NaN,NaN,NaN,8.000000,NaN,NaN,NaN,NaN,2012.000000,4.000000
75%,101795.250000,NaN,NaN,2.933408,47.000000,179.000000,78.400000,NaN,NaN,NaN,...,NaN,NaN,NaN,15.000000,NaN,NaN,NaN,NaN,2012.000000,8.000000
